In [6]:
# load libraries
#%matplotlib inline
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle


# List of FIPS state ids
states = ['01', '03', '04', '05', '06', '08', '09', '10', '11', '12', '13', '14', '16', '17', '18', '19', '20', 
         '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', 
         '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53',
          '54', '55', '56']

#       ,'02', '15'] 

# Dictionary connecting state names and FIPS ids
states_dict = {'Alabama':'01', 'Arizona':'03', 'Arkansas':'05', 'California':'06', 'Colorado':'08', 
              'Connecticut':'09', 'Delaware':'10', 'District of Columbia':'11', 'Florida':'12',
              'Georgia':'13', 'Idaho':'16', 'Illinois':'17', 'Indiana':'18', 'Iowa':'19', 'Kansas':'20',
              'Kentucky':'21', 'Louisiana':'22', 'Maine':'23', 'Maryland':'24', 'Massachusetts':'25',
              'Michigan':'26', 'Minnesota':'27', 'Mississippi':'28', 'Missouri':'29', 'Montana':'30',
              'Nebraska':'31', 'Nevada':'32', 'New Hampshire':'33', 'New Jersey':'34', 'New Mexico':'35',
              'New York':'36', 'North Carolina':'37', 'North Dakota':'38', 'Ohio':'39', 'Oklahoma':'40',
              'Oregon':'41', 'Pennsylvania':'42', 'Rhode Island':'44', 'South Carolina':'45', 
              'South Dakota':'46', 'Tennessee':'47', 'Texas':'48', 'Utah':'49', 'Vermont':'50',
              'Virginia':'51', 'Washington':'53', 'West Virginia':'54', 'Wisconsin':'55', 'Wyoming':'56'}

#              'Alaska':'02', 'Hawaii':'15'}

In [7]:
# Provide location of data files
#geo_fp = '../data_geographic/geojson-counties-fips.json'
#pop_fp = '../data_population/co-est2019-alldata.csv'
#covid_fp =  '../NYT-covid-19-data/us-counties.csv'


In [8]:
# Read in gdf with geo and pop data from pickle file

with open('us_county_geo_pop_gdf.pickle', 'rb') as f:
    gdf = pickle.load(f)

#gdf

gdf[gdf['fips']=='53061']

,fips,state_code,county_code,county_name,geometry,population
2949,53061,53,061,Snohomish,"MULTIPOLYGON (((-122.32172 48.01998, -122.3034...",822083


In [9]:
# Read in COVID data from NYT repo
df=pd.read_csv(covid_fp, dtype={'fips':'str'})

# Drop non-county data with no FIPS
df=df.dropna(subset=['fips'])
df=df[df['state']=='Washington']

# Trim unwanted columns
df = df[['date','fips','cases','deaths']]

# Create a list of dates for which we have data
dates = df['date'].tolist()

# Remove any duplicate dates
dates = list(dict.fromkeys(dates))


# Next, we loop over all the dates for which we have data, and merge that data with the gdf
for d in dates:
    # Create a new temporary dataframe for the current date
    # this contains the current date, fips, cases and death data
    df_ = df[df['date']==d]
    
    # Make a list of the case and death data
    cases = df_['cases'].tolist()
    deaths = df_['deaths'].tolist()
    
    # Create a list of dictionaries which contain the case and death data
    # so for each fips, for the current date, we have a dictionary with two pieces of data --- more can be added
    data = [{'cases':cases[i], 'deaths':deaths[i]} for i in range(len(cases))]
    
    # Tack the data to the end of the temporary data frame
    df_.insert(loc=4,column=d, value=data)
    
    # Merge df_ with gdf
    gdf = gdf.merge(df_[['fips',d]], on='fips', how='left')
    
    # If a fips (location) does not have data for the current date, that cell will have a NA.
    # Check for any cells with an 'NA' and replace them with a zero. 
    gdf[d].fillna(0, inplace=True)

# Dump the merged gdf to a pickle file
# Dump data to pickle file
with open('us_county_geo_pop_covid_gdf.pickle', 'wb') as f:
    pickle.dump(gdf, f, pickle.HIGHEST_PROTOCOL)

gdf

,fips,state_code,county_code,county_name,geometry,population,2020-01-21,2020-01-22,2020-01-23,2020-01-24,...,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29
0,01001,01,001,Autauga,"POLYGON ((-86.49677 32.34444, -86.71790 32.402...",55869,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01003,01,003,Baldwin,"POLYGON ((-87.59893 30.99745, -87.59411 30.976...",223234,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,01005,01,005,Barbour,"POLYGON ((-85.05603 32.06305, -85.05021 32.024...",24686,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,01007,01,007,Bibb,"POLYGON ((-87.42120 32.87451, -87.42013 32.902...",22394,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,01009,01,009,Blount,"POLYGON ((-86.57780 33.76532, -86.75914 33.840...",57826,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,56037,56,037,Sweetwater,"POLYGON ((-110.04800 41.57802, -110.05371 42.2...",42343,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3104,56039,56,039,Teton,"POLYGON ((-111.04875 44.06040, -111.04875 44.0...",23464,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3105,56041,56,041,Uinta,"POLYGON ((-111.04640 41.57984, -110.06279 41.5...",20226,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3106,56043,56,043,Washakie,"POLYGON ((-108.55076 44.07940, -108.55056 44.1...",7805,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#gdf[gdf['fips']=='53033']
#gdf.iloc[2935:2940,5].tolist()

# This displays the rows 2935-2945 in gdf. Which turn out to be a bunch of counties here in Washington
gdf[2935:2945]

,fips,state_code,county_code,county_name,geometry,population,2020-01-21,2020-01-22,2020-01-23,2020-01-24,...,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29
2935,53033,53,033,King,"MULTIPOLYGON (((-122.48499 47.51273, -122.4678...",2252782,0,0,0,0,...,"{'cases': 9167, 'deaths': 603}","{'cases': 9236, 'deaths': 602}","{'cases': 9275, 'deaths': 604}","{'cases': 9371, 'deaths': 606}","{'cases': 9504, 'deaths': 606}","{'cases': 9614, 'deaths': 608}","{'cases': 9697, 'deaths': 609}","{'cases': 9821, 'deaths': 612}","{'cases': 9903, 'deaths': 612}","{'cases': 10071, 'deaths': 612}"
2936,53035,53,035,Kitsap,"POLYGON ((-122.74920 47.74106, -122.74016 47.7...",271473,0,0,0,0,...,"{'cases': 189, 'deaths': 2}","{'cases': 190, 'deaths': 2}","{'cases': 191, 'deaths': 2}","{'cases': 195, 'deaths': 2}","{'cases': 198, 'deaths': 2}","{'cases': 200, 'deaths': 2}","{'cases': 205, 'deaths': 2}","{'cases': 211, 'deaths': 2}","{'cases': 213, 'deaths': 2}","{'cases': 212, 'deaths': 2}"
2937,53037,53,037,Kittitas,"POLYGON ((-121.24321 47.54033, -121.11361 47.5...",47935,0,0,0,0,...,"{'cases': 95, 'deaths': 0}","{'cases': 98, 'deaths': 0}","{'cases': 99, 'deaths': 0}","{'cases': 100, 'deaths': 0}","{'cases': 109, 'deaths': 0}","{'cases': 112, 'deaths': 0}","{'cases': 113, 'deaths': 0}","{'cases': 117, 'deaths': 0}","{'cases': 122, 'deaths': 0}","{'cases': 127, 'deaths': 0}"
2938,53039,53,039,Klickitat,"POLYGON ((-121.60858 45.85822, -121.61250 46.0...",22425,0,0,0,0,...,"{'cases': 51, 'deaths': 3}","{'cases': 51, 'deaths': 3}","{'cases': 52, 'deaths': 3}","{'cases': 53, 'deaths': 3}","{'cases': 55, 'deaths': 3}","{'cases': 55, 'deaths': 3}","{'cases': 56, 'deaths': 3}","{'cases': 57, 'deaths': 3}","{'cases': 59, 'deaths': 3}","{'cases': 60, 'deaths': 3}"
2939,53041,53,041,Lewis,"POLYGON ((-121.52356 46.38822, -122.24101 46.3...",80707,0,0,0,0,...,"{'cases': 47, 'deaths': 3}","{'cases': 49, 'deaths': 3}","{'cases': 49, 'deaths': 3}","{'cases': 50, 'deaths': 3}","{'cases': 52, 'deaths': 3}","{'cases': 53, 'deaths': 3}","{'cases': 57, 'deaths': 3}","{'cases': 60, 'deaths': 3}","{'cases': 64, 'deaths': 3}","{'cases': 64, 'deaths': 3}"
2940,53043,53,043,Lincoln,"POLYGON ((-117.82078 47.82519, -117.82370 47.2...",10939,0,0,0,0,...,"{'cases': 2, 'deaths': 0}","{'cases': 2, 'deaths': 0}","{'cases': 2, 'deaths': 0}","{'cases': 2, 'deaths': 0}","{'cases': 2, 'deaths': 0}","{'cases': 2, 'deaths': 0}","{'cases': 2, 'deaths': 0}","{'cases': 3, 'deaths': 0}","{'cases': 3, 'deaths': 0}","{'cases': 3, 'deaths': 0}"
2941,53045,53,045,Mason,"POLYGON ((-122.90566 47.15561, -122.93672 47.1...",66768,0,0,0,0,...,"{'cases': 42, 'deaths': 1}","{'cases': 42, 'deaths': 1}","{'cases': 43, 'deaths': 1}","{'cases': 43, 'deaths': 1}","{'cases': 43, 'deaths': 1}","{'cases': 46, 'deaths': 1}","{'cases': 45, 'deaths': 1}","{'cases': 46, 'deaths': 1}","{'cases': 47, 'deaths': 1}","{'cases': 47, 'deaths': 1}"
2942,53047,53,047,Okanogan,"POLYGON ((-120.85120 49.00119, -119.70202 49.0...",42243,0,0,0,0,...,"{'cases': 82, 'deaths': 2}","{'cases': 82, 'deaths': 2}","{'cases': 82, 'deaths': 2}","{'cases': 82, 'deaths': 2}","{'cases': 88, 'deaths': 2}","{'cases': 88, 'deaths': 2}","{'cases': 90, 'deaths': 2}","{'cases': 93, 'deaths': 2}","{'cases': 93, 'deaths': 2}","{'cases': 93, 'deaths': 2}"
2943,53049,53,049,Pacific,"POLYGON ((-123.35825 46.38423, -123.72606 46.3...",22471,0,0,0,0,...,"{'cases': 12, 'deaths': 1}","{'cases': 12, 'deaths': 1}","{'cases': 13, 'deaths': 1}","{'cases': 13, 'deaths': 1}","{'cases': 13, 'deaths': 1}","{'cases': 12, 'deaths': 1}","{'cases': 12, 'deaths': 1}","{'cases': 12, 'deaths': 1}","{'cases': 13, 'deaths': 1}","{'cases': 13, 'deaths': 1}"
2944,53051,53,051,Pend Oreille,"POLYGON ((-117.42954 48.99996, -117.26819 48.9...",13724,0,0,0,0,...,"{'cases': 3, 'deaths': 0}","{'cases': 3, 'deaths': 0}","{'cases': 3, 'deaths': 0}","{'cases': 3, 'deaths': 0}","{'cases': 3, 'deaths': 0}","{'cases': 3, 'deat

In [11]:
# This cell demonstrates one way to grab multiple columns from a dataframe by sending a list of the column
# names to the datframe

titles = ['population', 'geometry']
#gdf[['population','geometry']]
gdf[titles]

,population,geometry
0,55869,"POLYGON ((-86.49677 32.34444, -86.71790 32.402..."
1,223234,"POLYGON ((-87.59893 30.99745, -87.59411 30.976..."
2,24686,"POLYGON ((-85.05603 32.06305, -85.05021 32.024..."
3,22394,"POLYGON ((-87.42120 32.87451, -87.42013 32.902..."
4,57826,"POLYGON ((-86.57780 33.76532, -86.75914 33.840..."
...,...,...
3103,42343,"POLYGON ((-110.04800 41.57802, -110.05371 42.2..."
3104,23464,"POLYGON ((-111.04875 44.06040, -111.04875 44.0..."
3105,20226,"POLYGON ((-111.04640 41.57984, -110.06279 41.5..."
3106,7805,"POLYGON ((-108.55076 44.07940, -108.55056 44.1..."


In [8]:
# This cell demonstrates another way to grab multiple columns from a dataframe using .iloc
gdf.iloc[:,5:7]

,population,2020-01-21
0,55869,0
1,223234,0
2,24686,0
3,22394,0
4,57826,0
...,...,...
3103,42343,0
3104,23464,0
3105,20226,0
3106,7805,0


In [9]:
# This cell shows one way to get the data for a single row given a value in that row
gdf[gdf['fips']=='53061']

,fips,state_code,county_code,county_name,geometry,population,2020-01-21,2020-01-22,2020-01-23,2020-01-24,...,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26
2949,53061,53,061,Snohomish,"MULTIPOLYGON (((-122.32172 48.01998, -122.3034...",822083,"{'cases': 1, 'deaths': 0}","{'cases': 1, 'deaths': 0}","{'cases': 1, 'deaths': 0}","{'cases': 1, 'deaths': 0}",...,"{'cases': 3641, 'deaths': 158}","{'cases': 3656, 'deaths': 159}","{'cases': 3678, 'deaths': 161}","{'cases': 3678, 'deaths': 162}","{'cases': 3678, 'deaths': 162}","{'cases': 3747, 'deaths': 163}","{'cases': 3813, 'deaths': 163}","{'cases': 3833, 'deaths': 164}","{'cases': 3862, 'deaths': 164}","{'cases': 3893, 'deaths': 166}"


In [12]:
gdf['fips']=='53061'

0       False
1       False
2       False
3       False
4       False
        ...  
3103    False
3104    False
3105    False
3106    False
3107    False
Name: fips, Length: 3108, dtype: bool